In [ ]:
import numpy as np # creating a short cut to acces numpy faster in future code
import matplotlib.pyplot as plt #importing matplotlib tool as a shortcut to be able to plot my conclusions. 
from numpy import loadtxt #importing a usefull tool from numpy

def read(file): #defining a new function to read in external files
    data = loadtxt(file) #calling the extracted file data
    data_new = data.transpose() # transposing the values, so I only got 3 rows of data and can acces it easier
    time = data_new[0] #the first row in this transposed data is the time the observations were made 
    flux = data_new[1] #the second row is the flux of the star
    error = data_new[2] #the last row is the error for the y-values(flux)
    
    return time, flux, error #returning three lists containing the unpacked values





time, flux, error = read("Transit_model_LC_follow-up_data.txt")

In [ ]:
t = np.linspace(time[0], time[-1], 2000)
import batman                         #importing the batman module to help plotting the transit curve


params = batman.TransitParams()       #object to store transit parameters
params.t0 = 3.725                      #time of inferior conjunction
params.per = 4.781                       #orbital period
params.rp = 0.08                       #planet radius (in units of stellar radii)
params.a = 10.                        #semi-major axis (in units of stellar radii)
params.inc = 89.5                      #orbital inclination (in degrees)
params.ecc = 0.                       #eccentricity
params.w = 0.                        #longitude of periastron (in degrees)
params.limb_dark = "nonlinear"        #limb darkening model
params.u = [0.5, 0.1, 0.1, -0.1]      #limb darkening coefficients [u1, u2, u3, u4]


m = batman.TransitModel(params, t)    #initializes model
flux_bat = m.light_curve(params)          #calculates light curve
 
plt.plot(time, flux,"o")
plt.errorbar(time, flux, fmt='.', yerr=error) #plotting the errorbar
plt.plot(t, flux_bat)
plt.title(f"Exoplanet", fontsize=40)
plt.xlabel('Time in Days', fontsize=20)
plt.ylabel('Flux ', fontsize=20)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()

In [ ]:
from scipy.optimize import minimize
import math

m = batman.TransitModel(params, time)
def fun(x0,a , b, c): #creating a function to be minimised
   
    params.t0 = x0[0]                        #time of inferior conjunction
    params.per = 4.781                       #orbital period
    params.rp = x0[1]                       #planet radius (in units of stellar radii)
    params.a = x0[2]                       #semi-major axis (in units of stellar radii)
    params.inc = x0[3]                      #orbital inclination (in degrees)
    params.ecc = 0.                       #eccentricity
    params.w = 0.                       #longitude of periastron (in degrees)
    params.limb_dark = "nonlinear"        #limb darkening model
    params.u = [0.5,0.1,0.1,-0.1]
    
    
    
    
    flux_bat = m.light_curve(params)
    if 0. < x0[0]  and   0.0 < x0[1] and 0.0 < x0[2] < 50 and 0.0 < x0[3] < 90  :
        return sum(((b-flux_bat)/c)**2)
    else:
         return float('inf')

    
         

    
                       

In [ ]:
x0 = [3.715,  0.081, 10.,87] 
m = batman.TransitModel(params, time)
res = minimize(fun, x0,args = (time, flux, error), method='Nelder-Mead', tol=1e-4 )# letting the otmize module run with x0 as initial guesse
x = res.x.tolist()
print(x)
true_value = res.x


In [ ]:
m = batman.TransitModel(params, t)
params = batman.TransitParams()       #object to store transit parameters
params.t0 = x[0]                        #time of inferior conjunction
params.per = 4.781                        #orbital period
params.rp = x[1]                       #planet radius (in units of stellar radii)
params.a = x[2]                        #semi-major axis (in units of stellar radii)
params.inc = x[3]                      #orbital inclination (in degrees)
params.ecc = 0.                       #eccentricity
params.w = 0.                       #longitude of periastron (in degrees)
params.limb_dark = "nonlinear"        #limb darkening model
params.u = [0.5,0.1,0.1,-0.1]

flux_bat3 = m.light_curve(params)   
plt.plot(time, flux,"o")
plt.errorbar(time, flux, fmt='.', yerr=error) #plotting the errorbar

plt.plot(t, flux_bat3)
plt.title(f"Exoplanet", fontsize=40)
plt.xlabel('Time in Days', fontsize=20)
plt.ylabel('Flux ', fontsize=20)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show() # cross checking the obtained values to the data

In [ ]:
#params = batman.TransitParams()       #object to store transit parameters
theta = x
m = batman.TransitModel(params, time)
def log_likelihood(theta, time_fun, flux_fun, error_fun): #defining a log likelihood function
    
    
    
    params.t0 = theta[0]                        #time of inferior conjunction
    params.per = 4.781                   #orbital period
    params.rp = theta[1]                      #planet radius (in units of stellar radii)
    params.a = theta[2]                        #semi-major axis (in units of stellar radii)
    params.inc = theta[3]                      #orbital inclination (in degrees)
    params.ecc = 0.                       #eccentricity
    params.w = 0.                       #longitude of periastron (in degrees)
    params.limb_dark = "nonlinear"        #limb darkening model
    params.u = [0.5,0.1,0.1,-0.1]  
    model = m.light_curve(params)
    
    
    
    return -0.5 * np.sum(((flux_fun - model) / error_fun)**2)


In [ ]:
def log_prior(theta): #defining a prior function
    
    if 0. < theta[0] <10 and 0.0 < theta[1] < 1 and 0.0 < theta[2] < 50 and 0.0 < theta[3] < 90  :
        return 0.0
  
        
    return -np.inf

In [ ]:
def log_probability(theta, time_fun, flux_fun, error_fun): #defining a log probability function
    lp = log_prior(theta)
    
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood(theta, time_fun, flux_fun, error_fun)

In [ ]:
import emcee

m = batman.TransitModel(params, time)
ndim = 4
nwalkers = 50
pos = true_value + 1e-4 * np.random.randn(50, 4)
nwalkers, ndim = pos.shape
sampler = emcee.EnsembleSampler(
    nwalkers, ndim, log_probability, args=( time, flux, error),
moves=emcee.moves.StretchMove())
sampler.run_mcmc(pos , 5000, progress=True); # letting the emcee module run 5000 times

In [ ]:
labels = ["Time(0)","Orbital" "Period","Planet"" Radius", "Semi-Major"" Axis" ,"Orbital"" Inclination"]#creating a list of labels for my plot
fig, axes = plt.subplots(4, figsize=(10, 7), sharex=True)
samples = sampler.get_chain()

for i in range(ndim): # printing the outcomes of the emcee sampler
    ax = axes[i]
    ax.plot(samples[:, :, i], "k", alpha=0.3)
    ax.set_xlim(0, len(samples))
    ax.set_ylabel(labels[i])

In [ ]:
import corner
labels = ["Time" "" "of" "Inferior" "Conjunction","Planet"" Radius", "Semi-Major"" Axis" ,"Orbital"" Inclination"]

flat_samples = sampler.get_chain( flat=True) # plotting corner plot of the results
fig = corner.corner(
    flat_samples,   truths= true_value, labels = labels
)

In [ ]:
from IPython.display import display, Math
results = []
result_uncertainty = []
for i in range(ndim):
    mcmc = np.percentile(flat_samples[:, i], [16, 50, 84]) # printing out the results with the corresponding uncertainties
    q = np.diff(mcmc)
    txt = "\mathrm{{{3}}} = {0:.3f}_{{-{1:.5f}}}^{{{2:.5f}}}"
    txt = txt.format(mcmc[1], q[0], q[1], labels[i])
    display(Math(txt))
    result_uncertainty.append([ mcmc[1], q[0], q[1]])
    results.append(mcmc[1])


In [ ]:

params = batman.TransitParams()       #object to store transit parameters
params.t0 = results[0]                     #time of inferior conjunction
params.per = 4.781                        #orbital period
params.rp = results[1]                       #planet radius (in units of stellar radii)
params.a = results[2]                        #semi-major axis (in units of stellar radii)
params.inc = results[3]                      #orbital inclination (in degrees)
params.ecc = 0.                       #eccentricity
params.w = 0.                       #longitude of periastron (in degrees)
params.limb_dark = "nonlinear"        #limb darkening model
params.u = [0.5,0.1,0.1,-0.1]
m = batman.TransitModel(params, t)    #initializes model
flux_bat = m.light_curve(params)   
plt.plot(time, flux,"o")
plt.errorbar(time, flux, fmt='.', yerr=error) #plotting the errorbar


plt.plot(t, flux_bat)
plt.title(f"Exoplanet", fontsize=40)
plt.xlabel('Time in Days', fontsize=20)
plt.ylabel('Flux ', fontsize=20)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()